In [ ]:
import os
import subprocess
import xml.etree.ElementTree as ET

GITHUB_TOKEN = ''
REPO_URL = f'https://github.com/karanghai9/db-fork-spring-boot.git'
LOCAL_DIR = 'forked_repo'
BRANCH_NAME = 'pom-upgrade'
COMMIT_MESSAGE = 'Upgrade pom.xml version'

# --- 1. Clone the repository ---
if not os.path.exists(LOCAL_DIR):
    subprocess.run(['git', 'clone', REPO_URL, LOCAL_DIR], check=True)

# --- 2. Create and switch to a new branch ---
# Check if the branch exists locally
branch = subprocess.run(
    ['git', 'branch', '--list', BRANCH_NAME],
    cwd=LOCAL_DIR,
    stdout=subprocess.PIPE,
    text=True
)
if branch.stdout.strip():
    print("Branch exists, checking it out.")
    subprocess.run(['git', 'checkout', BRANCH_NAME], cwd=LOCAL_DIR, check=True)
else:
    print("Branch does not exist, creating it.")
    subprocess.run(['git', 'checkout', '-b', BRANCH_NAME], cwd=LOCAL_DIR, check=True)

pom_path = os.path.join(os.getcwd(), 'forked_repo', 'pom.xml')
new_version = '3.0.3'

# Namespace used in your pom.xml
NAMESPACE = "http://maven.apache.org/POM/4.0.0"

# Register the default namespace (empty string) to avoid ns0: prefixes
ET.register_namespace('', NAMESPACE)

tree = ET.parse(pom_path)
root = tree.getroot()

version_element = root.find(f'{{{NAMESPACE}}}version')
if version_element is not None:
    version_element.text = new_version

    # Write XML WITHOUT declaration to a temp file or string
    import io
    temp_io = io.BytesIO()
    tree.write(temp_io, encoding='UTF-8', xml_declaration=False)
    xml_body = temp_io.getvalue()

    # Now write final file with your exact desired declaration manually prepended
    with open(pom_path, 'wb') as f:
        f.write(b'<?xml version="1.0" encoding="UTF-8"?>\n')
        f.write(xml_body)

    print(f"✅ Updated version to {new_version} with exact XML declaration")
else:
    print("⚠️ <version> tag not found")

# Save file
tree.write(pom_path, encoding='utf-8', xml_declaration=True)

# Push changes
subprocess.run(['git', 'add', 'pom.xml'], cwd=LOCAL_DIR, check=True)
subprocess.run(['git', 'commit', '-m', COMMIT_MESSAGE], cwd=LOCAL_DIR,  check=True)

# Use HTTPS with GitHub token (replace before running or use SSH instead)
subprocess.run(['git', 'push', 'origin', BRANCH_NAME], cwd=LOCAL_DIR, check=True)
print(f"Changes committed to branch '{BRANCH_NAME}'. Push manually if needed.")

Cloning into 'forked_repo'...


Branch does not exist, creating it.
✅ Updated version to 3.0.3 with exact XML declaration
[pom-upgrade ed544424cf] Upgrade pom.xml version
 1 file changed, 11 insertions(+), 15 deletions(-)


Switched to a new branch 'pom-upgrade'


Changes committed to branch 'pom-upgrade'. Push manually if needed.


remote: 
remote: Create a pull request for 'pom-upgrade' on GitHub by visiting:        
remote:      https://github.com/karanghai9/db-fork-spring-boot/pull/new/pom-upgrade        
remote: 
To https://github.com/karanghai9/db-fork-spring-boot.git
 * [new branch]            pom-upgrade -> pom-upgrade
